In [4]:
import csv
reader = csv.reader(open("wind-farm-1-signals-2016.csv", "r"), delimiter=';')
writer = csv.writer(open("output.csv", 'w'), delimiter=',')
writer.writerows(reader)


In [1]:
import pandas as pd 
df = pd.read_csv('output.csv')
df2 = pd.DataFrame(df)
df2.drop(['Turbine_ID'], axis = 1,inplace=True)
df2.drop(['Timestamp'], axis = 1,inplace=True)
start_test = '2016-02-26T15:30:00+00:00'
train, test = df2.loc[:start_test],df2.loc[start_test:] 




df2



,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,Hyd_Oil_Temp_Avg,Gear_Oil_Temp_Avg,...,Grd_Prod_PsbleInd_Avg,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg
0,1634.3,1226.9,1416.4,96.4,49,62,63,64,30,52,...,-998.7,-649.2,-1000.0,18.5,998.7,1000.0,649.2,18.5,40,207.1
1,1796.1,1597.0,1680.0,29.8,66,97,107,111,43,54,...,-615.2,-583.3,-1000.0,91.9,423.8,783.5,406.1,52.7,57,129.9
2,1657.5,1299.2,1495.0,83.8,47,62,62,61,29,49,...,-1000.0,-1000.0,-1000.0,0.0,1000.0,1000.0,1000.0,0.0,49,220.6
3,1771.2,1590.0,1677.5,29.4,66,111,110,110,50,59,...,-639.6,-583.3,-1000.0,132.4,452.2,1000.0,406.1,125.1,56,123.5
4,1667.5,1277.4,1481.3,119.1,46,73,72,71,30,57,...,-1000.0,-1000.0,-1000.0,0.0,998.6,1000.0,884.5,11.1,47,206.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207900,1555.5,1194.1,1301.3,75.6,48,60,61,58,34,48,...,-891.2,-143.3,-1000.0,168.7,891.2,1000.0,143.3,168.7,41,174.4
207901,1642.0,1240.3,1410.9,107.7,45,66,66,65,32,51,...,-990.1,-561.9,-1000.0,53.5,990.1,1000.0,561.9,53.5,42,172.1
207902,1672.4,1305.5,1563.9,92.9,46,75,74,73,30,55,...,-1000.0,-1000.0,-1000.0,0.0,1000.0,1000.0,1000.0,0.0,54,184.5
207903,1659.8,1211.2,1346.5,115.7,48,60,61,58,34,49,...,-960.4,-508.6,-1000.0,95.6,960.4,1000.0,508.6,95.6,42,171.3


In [2]:
from sklearn.preprocessing import MinMaxScaler
SCALER = MinMaxScaler(feature_range=(-1,1))
scaler = SCALER.fit(train.to_numpy())
train_scaled = scaler.transform(train.to_numpy())
test_scaled = scaler.transform(test.to_numpy())

In [3]:
import numpy as np
timestep = 24*7*2 # 24hours,7days,2weeks
def create_dataset(dataset, timestep=timestep):
    
    X, y = [], []
    for i in range(len(dataset)):
        target_value = i + timestep
        if target_value == len(dataset):
            break
        feature_chunk, target = dataset[i:target_value, 1:],   dataset[target_value, 0]
        X.append(feature_chunk)
        y.append(target)
    
    return np.array(X), np.array(y)

In [ ]:
X_train, y_train = create_dataset(train_scaled)
X_test, y_test = create_dataset(test_scaled)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
import keras
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf 

def create_model(X_train, y_train):
        
        units = 32
        dropout = 0.05
        epochs = 35
        batch_size = 14
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
        early_stopping = EarlyStopping(patience=7, monitor='loss')
        model = keras.Sequential()
        model.add(LSTM(units=units, dropout=dropout,      return_sequences=True,input_shape=(X_train.shape[1],    X_train.shape[2])))
        
        model.add(LSTM(units=units, dropout=dropout))
        
        model.add(Dense(units=1))
        model.compile(optimizer=optimizer,loss='mean_squared_error')
        
        history = model.fit(X_train, y_train, validation_split=0.3, shuffle=False, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[early_stopping])
       
        return model, history

In [ ]:
def single_prediction(model, history, timestep=timestep):
        
        history = np.array(history)
        history = history.reshape(history.shape[0]*history.shape[1],   history.shape[2])
        
        input_value = history[-timestep:]
        input_value = input_value.reshape(1, input_value.shape[0], input_value.shape[1])
        
        yhat = model.predict(input_value, verbose=0)
        return yhat

In [31]:
def walk_forward_prediction(X_train, y_train, X_test, timestep):
    
    MODEL, history = create_model(X_train=X_train, y_train=y_train)
    hist_train = [i for i in X_train]
    predictions = []
    
    for i in range(len(X_test)):
        test = X_test[i]
        yhat = single_prediction(model=MODEL, history=hist_train, timestep=timestep)
        predictions.append(yhat) 
        hist_train.append(test)
    
    return predictions, history, MODEL

In [32]:
def prior_inverse(features, targets):
   
    dataset = []
    
    for i in range(features.shape[0]):
        last_row, target = features[i][0], targets[i]
        appended = np.append(last_row, target)
        dataset.append(appended)
    
    return np.array(dataset)

In [33]:
def experiment(X_train, y_train, X_test, timestep):
    
    pred_seq, history, MODEL = walk_forward_prediction(X_train, y_train, X_test, timestep)
    
    pred_seq = np.array(pred_seq).reshape(-1)
    pred = prior_inverse(X_test, pred_seq)
    real = prior_inverse(X_test, y_test)
    inv_pred = scaler.inverse_transform(pred)
    inv_real = scaler.inverse_transform(real)
    power_pred = inv_pred[:,-1]
    power_real = inv_real[:,-1]
    
    return power_real, power_pred, history, MODEL

In [ ]:
power_real, power_pred, history, MODEL = experiment(X_train, y_train, X_test, timestep)
loss = history.history['loss']
val_loss = history.history['val_loss']

Epoch 1/35
2288/2288 [==============================] - 347s 150ms/step - loss: 0.0975 - val_loss: 0.0518
Epoch 2/35
2288/2288 [==============================] - 339s 148ms/step - loss: 0.0817 - val_loss: 0.0440
Epoch 3/35
2288/2288 [==============================] - 342s 150ms/step - loss: 0.0783 - val_loss: 0.0431
Epoch 4/35
2288/2288 [==============================] - 484s 211ms/step - loss: 0.0708 - val_loss: 0.0397
Epoch 5/35
2288/2288 [==============================] - 354s 155ms/step - loss: 0.0763 - val_loss: 0.0374
Epoch 6/35
2288/2288 [==============================] - 334s 146ms/step - loss: 0.0748 - val_loss: 0.0398
Epoch 7/35
2288/2288 [==============================] - 336s 147ms/step - loss: 0.0731 - val_loss: 0.0351
Epoch 8/35
2288/2288 [==============================] - 336s 147ms/step - loss: 0.0757 - val_loss: 0.0367
Epoch 9/35
2288/2288 [==============================] - 340s 149ms/step - loss: 0.0773 - val_loss: 0.0349
Epoch 10/35
2288/2288 [=======================